# Download Packages

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-86163367-94e2-1d3b-ede1-17dd7215dbeb)
GPU 1: Tesla T4 (UUID: GPU-ba3c2cc9-fb8b-d1fb-d06c-67d029b4b1d7)


In [1]:
%%capture
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install mne #The MNE Package is installed
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git
!dir

In [2]:
!apt-get install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 -y
!pip install tensorflow==2.8.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libcudnn8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.9/497.9 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 45.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.2
    Uninstalling tensorboard-2.11.2:
      Successfully uninstalled tensorb

# Functions

In [3]:
from gcpds.databases.BCI_Competition_IV import Dataset_2a
from typing import Sequence, Tuple
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
import numpy as np
from scipy.signal import resample

def load_BCICIV2a(db: Dataset_2a,
               sbj: int,
               mode: str,
               fs: float, 
               f_bank: np.ndarray, 
               vwt: np.ndarray, 
               new_fs: float) -> np.ndarray:

  tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

  db.load_subject(sbj, mode = mode)
  X, y = db.get_data() #Load all classes, all channels {EEG, EOG}, reject bad trials
  X = X[:,:-3,:] # pick EEG channels
  X = X*1e6 #uV
  X = np.squeeze(tf_repr.transform(X))
  #Resampling
  if new_fs == fs:
    print('No resampling, since new sampling rate same.')
  else:
    print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
    X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    
  return X, y


from gcpds.databases import GIGA_MI_ME

def load_GIGA_MI_ME(db: GIGA_MI_ME,
              sbj: int,
              eeg_ch_names: Sequence[str],
              fs: float, 
              f_bank: np.ndarray, 
              vwt: np.ndarray, 
              new_fs: float) -> Tuple[np.ndarray, np.ndarray]:

  index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

  tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

  db.load_subject(sbj)
  X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
  X = X[:, index_eeg_chs, :] #spatial rearrangement
  X = np.squeeze(tf_repr.transform(X))
  #Resampling
  if new_fs == fs:
    print('No resampling, since new sampling rate same.')
  else:
    print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
    X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    
  return X, y


def load_DB(db_name, **load_args):
  if db_name == 'BCICIV2a':
    X_train, y_train = load_BCICIV2a(**load_args, mode = 'training')
    X_test, y_test = load_BCICIV2a(**load_args, mode = 'evaluation')

    X_train = np.concatenate([X_train, X_test], axis = 0)
    y_train = np.concatenate([y_train, y_test], axis = 0)

  elif db_name == 'GIGA_MI_ME':
    X_train, y_train = load_GIGA_MI_ME(**load_args)

  else:
    raise ValueError('No valid database name')

  return X_train, y_train


from EEG_Tensorflow_models.Models import DeepConvNet, ShallowConvNet, EEGNet, DMTL_BCI, TCNet_fusion, PST_attention


def get_model(model_name, nb_classes):
  if model_name == 'DeepConvNet':
    model = DeepConvNet
    model_params = dict(nb_classes = nb_classes,
                      dropoutRate = 0.5, version='2018')
    
  elif model_name == 'ShallowConvNet':
    model = ShallowConvNet
    model_params = dict(nb_classes = nb_classes,
                      dropoutRate = 0.5,
                      version = '2018')
    
  elif model_name == 'EEGNet':
    model = EEGNet
    model_params = dict(nb_classes = nb_classes,
                      dropoutRate = 0.5,
                      kernLength = 32,
                      F1 = 8,
                      D = 2,
                      F2 = 16,
                      norm_rate = 0.25,
                      dropoutType = 'Dropout')
    
  elif model_name == 'DMTL_BCI':
    model = DMTL_BCI
    model_params = dict(nb_classes = nb_classes,
                      dropoutRate = 0.5,
                      l1 = 0,
                      l2 = 0)
    
  elif model_name == 'TCNet_fusion':
    model = TCNet_fusion
    model_params = dict(nb_classes = nb_classes,
                      layers = 2,
                      kernel_s = 4,
                      filt = 12,
                      dropout = 0.3,
                      activation = 'relu',
                      F1 = 24,
                      D = 2,
                      kernLength = 32,
                      N_residuals = 2)
    
  elif model_name == 'PST_attention':
    model = PST_attention
    model_params = dict(nb_classes = nb_classes,
                      dropoutRate = 0.5,
                      last_layer = 'Dense')
    
  else:
    raise ValueError('No valid model name')
    
  return model, model_params

from tensorflow.random import set_seed
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score,\
                            f1_score, recall_score, precision_score

def train(db_name, load_args, cv_args, model_args, compile_args, fit_args, seed):
    X_train, y_train = load_DB(db_name, **load_args)
    X_train = X_train[..., np.newaxis]
    
    cv_results = {'params': [],
                  'mean_acc': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_kappa': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_auc': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_f1_left': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_f1_right': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_recall_left': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_recall_right': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_precision_left': np.zeros(cv_args['cv'].get_n_splits()),
                  'mean_precision_right': np.zeros(cv_args['cv'].get_n_splits()),}
    
    if model_args['nb_classes'] == 4:
        cv_results['mean_f1_legs'] = np.zeros(cv_args['cv'].get_n_splits())
        cv_results['mean_f1_tongue'] = np.zeros(cv_args['cv'].get_n_splits())
        cv_results['mean_recall_legs'] = np.zeros(cv_args['cv'].get_n_splits())
        cv_results['mean_recall_tongue'] = np.zeros(cv_args['cv'].get_n_splits())
        cv_results['mean_precision_legs'] = np.zeros(cv_args['cv'].get_n_splits())
        cv_results['mean_precision_tongue'] = np.zeros(cv_args['cv'].get_n_splits())

    k = 0
    max_acc = -np.inf
    for train_index, val_index in cv_args['cv'].split(X_train, y_train):
      X, X_val = X_train[train_index], X_train[val_index]
      y, y_val = y_train[train_index], y_train[val_index]
      print(val_index)

      if model_args['autoencoder']:
        y = [X, y]

      batch_size, C, T = X.shape[:-1]

      clear_session()
      set_seed(seed)

      model_cll, model_params = get_model(model_args['model_name'], model_args['nb_classes'])
      model = model_cll(**model_params, Chans = C, Samples = T)
      model.compile(loss = compile_args['loss'], 
                    optimizer = Adam(compile_args['init_lr']))
      
      history = model.fit(X, y,
                batch_size = batch_size,
                **fit_args)

      if model_args['autoencoder']:
        y_prob = model.predict(X_val)[-1]
        y_pred = np.argmax(y_prob, axis = 1)
      else:
        y_prob = model.predict(X_val)
        y_pred = np.argmax(y_prob, axis = 1)

      cv_results['mean_acc'][k] = accuracy_score(y_val, y_pred)
      cv_results['mean_kappa'][k] = cohen_kappa_score(y_val, y_pred)
      if model_args['nb_classes'] == 2:
        cv_results['mean_auc'][k] = roc_auc_score(y_val, y_prob[:, 1], average = 'macro')
        cv_results['mean_f1_left'][k] = f1_score(y_val, y_pred, pos_label = 0, average = 'binary')
        cv_results['mean_f1_right'][k] = f1_score(y_val, y_pred, pos_label = 1, average = 'binary')
        cv_results['mean_recall_left'][k] = recall_score(y_val, y_pred, pos_label = 0, average = 'binary')
        cv_results['mean_recall_right'][k] = recall_score(y_val, y_pred, pos_label = 1, average = 'binary')
        cv_results['mean_precision_left'][k] = precision_score(y_val, y_pred, pos_label = 0, average = 'binary')
        cv_results['mean_precision_right'][k] = precision_score(y_val, y_pred, pos_label = 1, average = 'binary')
      else:                                                                                  
        cv_results['mean_auc'][k] = roc_auc_score(y_val, y_prob, average = 'macro', multi_class = 'ovo')
        
        cv_results['mean_f1_left'][k] = f1_score(y_val, y_pred, pos_label = 0, average = 'micro')
        cv_results['mean_f1_right'][k] = f1_score(y_val, y_pred, pos_label = 1, average = 'micro')
        cv_results['mean_f1_legs'][k] = f1_score(y_val, y_pred, pos_label = 2, average = 'micro')
        cv_results['mean_f1_tongue'][k] = f1_score(y_val, y_pred, pos_label = 3, average = 'micro')
        cv_results['mean_recall_left'][k] = recall_score(y_val, y_pred, pos_label = 0, average = 'micro')
        cv_results['mean_recall_right'][k] = recall_score(y_val, y_pred, pos_label = 1, average = 'micro')
        cv_results['mean_recall_legs'][k] = recall_score(y_val, y_pred, pos_label = 2, average = 'micro')
        cv_results['mean_recall_tongue'][k] = recall_score(y_val, y_pred, pos_label = 3, average = 'micro')
        cv_results['mean_precision_left'][k] = precision_score(y_val, y_pred, pos_label = 0, average = 'micro')
        cv_results['mean_precision_right'][k] = precision_score(y_val, y_pred, pos_label = 1, average = 'micro')
        cv_results['mean_precision_legs'][k] = precision_score(y_val, y_pred, pos_label = 2, average = 'micro')
        cv_results['mean_precision_tongue'][k] = precision_score(y_val, y_pred, pos_label = 3, average = 'micro')
                                                       
                                                       
      if cv_results['mean_acc'][k]  > max_acc:
        max_acc = cv_results['mean_acc'][k]
        model.save_weights('sbj' + str(load_args['sbj']) +'.h5')

      k += 1
                                                
    cv_results['std_acc'] = round(cv_results['mean_acc'].std(), 3)
    cv_results['mean_acc'] = round(cv_results['mean_acc'].mean(), 3)
    cv_results['std_kappa'] = round(cv_results['mean_kappa'].std(), 3)
    cv_results['mean_kappa'] = round(cv_results['mean_kappa'].mean(), 3)
    cv_results['std_auc'] = round(cv_results['mean_auc'].std(), 3)
    cv_results['mean_auc'] = round(cv_results['mean_auc'].mean(), 3)
      
    cv_results['mean_f1_left'] = round(cv_results['mean_f1_left'].mean(), 3)
    cv_results['std_f1_left'] = round(cv_results['mean_f1_left'].std(), 3)
    cv_results['mean_f1_right'] = round(cv_results['mean_f1_right'].mean(), 3)
    cv_results['std_f1_right'] = round(cv_results['mean_f1_right'].std(), 3)
    cv_results['mean_recall_left'] = round(cv_results['mean_recall_left'].mean(), 3)
    cv_results['std_recall_left'] = round(cv_results['mean_recall_left'].std(), 3)
    cv_results['mean_recall_right'] = round(cv_results['mean_recall_right'].mean(), 3)
    cv_results['std_recall_right'] = round(cv_results['mean_recall_right'].std(), 3)
    cv_results['mean_precision_left'] = round(cv_results['mean_precision_left'].mean(), 3)
    cv_results['std_precision_left'] = round(cv_results['mean_precision_left'].std(), 3)
    cv_results['mean_precision_right'] = round(cv_results['mean_precision_right'].mean(), 3)
    cv_results['std_precision_right'] = round(cv_results['mean_precision_right'].std(), 3)

    if model_args['nb_classes'] == 4:
        cv_results['mean_f1_legs'] = round(cv_results['mean_f1_legs'].mean(), 3)
        cv_results['std_f1_legs'] = round(cv_results['mean_f1_legs'].std(), 3)
        cv_results['mean_f1_tongue'] = round(cv_results['mean_f1_tongue'].mean(), 3)
        cv_results['std_f1_tongue'] = round(cv_results['mean_f1_tongue'].std(), 3)
        cv_results['mean_recall_legs'] = round(cv_results['mean_recall_legs'].mean(), 3)
        cv_results['std_recall_legs'] = round(cv_results['mean_recall_legs'].std(), 3)
        cv_results['mean_recall_tongue'] = round(cv_results['mean_recall_tongue'].mean(), 3)
        cv_results['std_recall_tongue'] = round(cv_results['mean_recall_tongue'].std(), 3)
        cv_results['mean_precision_legs'] = round(cv_results['mean_precision_legs'].mean(), 3)
        cv_results['std_precision_legs'] = round(cv_results['mean_precision_legs'].std(), 3)
        cv_results['mean_precision_tongue'] = round(cv_results['mean_precision_tongue'].mean(), 3)
        cv_results['std_precision_tongue'] = round(cv_results['mean_precision_tongue'].std(), 3)
    
    return cv_results

# Config

In [4]:
# Marcos, use these two variables to run the state of the art. First, for BCICIV2a run all the models.
# Remeber that this network DMTL_BCI is an autoencoder. Set the nb_classses parameter depending of the database.
# set autoencoder based on the model
# We need to run all these tests again. Do not forget to add the recall, preci, and f1 for each class (bci 4, giga 2)
db_name = 'GIGA_MI_ME'
model_args = dict(model_name = 'EEGNet',
                  nb_classes = 2,
                  autoencoder = False)

In [5]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
import numpy as np
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from sklearn.model_selection import StratifiedShuffleSplit

if db_name == 'BCICIV2a':
  db = Dataset_2a('/kaggle/input/dataset-2a')
  fs = db.metadata['sampling_rate']
  load_args = dict(db = db,
                 fs = fs,
                 f_bank = np.asarray([[4., 40.]]),
                 vwt = np.asarray([[2.5, 6]]),
                 new_fs = 128.)
  subjects = np.arange(db.metadata['subjects']) + 1
  
elif db_name == 'GIGA_MI_ME':
  db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
  fs = db.metadata['sampling_rate']
  eeg_ch_names = ['Fp1','Fpz','Fp2',
                'AF7','AF3','AFz','AF4','AF8',
                'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
                'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
                'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
                'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
                'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
                'PO7','PO3','POz','PO4','PO8',
                'O1','Oz','O2',
                'Iz']

  load_args = dict(db = db,
                  eeg_ch_names = eeg_ch_names,
                  fs = fs,
                  f_bank = np.asarray([[4., 40.]]),
                  vwt = np.asarray([[2.5, 5]]),
                  new_fs = 128.)
  subjects = np.arange(db.metadata['subjects']) + 1
  subjects = np.delete(subjects, [28,33])
  
else:
  raise ValueError('No valid database name')

verbose = 0
reduce_lr_on_plateau = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 30, verbose = verbose, mode = 'min', min_delta = 0.01, min_lr = 0)
terminate_on_nan = TerminateOnNaN()
callbacks = [reduce_lr_on_plateau, terminate_on_nan]
seed = 23

cv_args = dict(cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = seed))

compile_args = dict(loss = SparseCategoricalCrossentropy(), #['mse' , SparseCategoricalCrossentropy()]
                    init_lr = 1e-2)
                      
fit_args = dict(epochs = 500,
                verbose = verbose,
                callbacks = callbacks)

# Main

In [ ]:
from pickle import dump

for sbj in subjects[:]:
  print('sbj = ', sbj)
  load_args['sbj'] = sbj
  results = train(db_name, load_args, cv_args, model_args, compile_args, fit_args, seed)
  with open('sbj' + str(load_args['sbj']) + '.txt', 'wb') as f:
    dump(results, f)

sbj =  1
Resampling from 512.000000 to 128.000000 Hz.
[ 25 107  31 188  83  86 111  75 191 170  76  12 100 154  73   6  66  54
 104 162  91  51 132  39 105  40 134 141  45 116 185 178 127 150 194  69
 133  90  64  49]
[175 156   7 191  76 117  54  16   2  80  41 118  78  90 100 160 149 161
 171  18  82 182 101  61  91 167 144 133 196  67  22  57 137 136  98 111
  89  13 178  97]


In [ ]:
!zip Models.zip ./*.h5
!zip Results.zip ./*.txt